![under_construction](figures/under_construction.gif)

### Riferimenti bibliografici:

* Hyvarinen, A. &  Oja E. (2012), [Independent Component Analysis: Algorithms and Applications](https://www.cs.helsinki.fi/u/ahyvarin/papers/NN00new.pdf).

# Analisi delle componenti indipendenti (ICA)

## Indice

1. [](#)<br />
    1.1 [](#)<br />

In [ ]:
from sklearn.decomposition import FastICA
from sklearn.pipeline import Pipeline
import inspect
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

# Problema del cocktail party

In [ ]:
from msbd.datasets import CocktailParty

print(inspect.getsource(CocktailParty))

In [ ]:
A = np.array(
    [[0.5, 0.5], 
     [0.2, 0.8]]
)

cp = CocktailParty(A)

In [ ]:
t = np.linspace(0, 100, 1000)
S = np.c_[cp.s1(t), cp.s2(t)]
X = np.c_[cp.x1(t), cp.x2(t)]

print("Dimensioni di S: {} X {}".format(*S.shape))
print("Dimensioni di X: {} X {}".format(*X.shape))

In [ ]:
from msbd.grafici import grafico_cocktail_party

print(inspect.getsource(grafico_cocktail_party))

In [ ]:
plt.figure(figsize=(8, 6))

grafico_cocktail_party(t, S, X)

plt.show()

In [ ]:
ica = FastICA(whiten=True, random_state=42)

S_hat = ica.fit_transform(X)

In [ ]:
plt.figure(figsize=(8, 8))

grafico_cocktail_party(t, S, X, S_hat)

plt.show()

### Esercizio

Elencare tutte le differenze tra segnali veri e stimati.

In [ ]:
from msbd.preprocessamento import Sbiancare


print(inspect.getsource(Sbiancare))

### Esercizio

1. Completare i metodo `inverse_transform()` della classe `Sbiancare` definita in `msbd/preprocessamento/sbiancare.py`;
2. Verificare il corretto funzionamento del metodo appena definito.

> Suggerimento: usare il fatto che la matrice $\mathbf{E}$ è una matrice [unitaria](https://it.wikipedia.org/wiki/Matrice_unitaria).

### Esercizio

1. Definire una pipeline `ica_pipe` con al primo step `Sbiancare()` e al secondo step `FastICA(whiten=False)`;
2. Ottenere `S_hat`come `S_hat = ica_pipe.fit_transform(X)`;
3. Fare nuovamente il grafico *cocktail party* utilizzando la stima di $\mathbf{S}$ ottenuta al passo precedente;
4. Commentare i risultati.

[Statlog (Landsat Satellite) Data Set](https://archive.ics.uci.edu/ml/datasets/Statlog+(Landsat+Satellite%29)

### Data Set Information:

The database consists of the multi-spectral values of pixels in 3x3 neighbourhoods in a satellite image, and the classification associated with the central pixel in each neighbourhood. The aim is to predict this classification, given the multi-spectral values. In the sample database, the class of a pixel is coded as a number. 

The Landsat satellite data is one of the many sources of information available for a scene. The interpretation of a scene by integrating spatial data of diverse types and resolutions including multispectral and radar data, maps indicating topography, land use etc. is expected to assume significant importance with the onset of an era characterised by integrative approaches to remote sensing (for example, NASA's Earth Observing System commencing this decade). Existing statistical methods are ill-equipped for handling such diverse data types. Note that this is not true for Landsat MSS data considered in isolation (as in this sample database). This data satisfies the important requirements of being numerical and at a single resolution, and standard maximum-likelihood classification performs very well. Consequently, for this data, it should be interesting to compare the performance of other methods against the statistical approach. 

One frame of Landsat MSS imagery consists of four digital images of the same scene in different spectral bands. Two of these are in the visible region (corresponding approximately to green and red regions of the visible spectrum) and two are in the (near) infra-red. Each pixel is a 8-bit binary word, with 0 corresponding to black and 255 to white. The spatial resolution of a pixel is about 80m x 80m. Each image contains 2340 x 3380 such pixels. 

The database is a (tiny) sub-area of a scene, consisting of 82 x 100 pixels. Each line of data corresponds to a 3x3 square neighbourhood of pixels completely contained within the 82x100 sub-area. Each line contains the pixel values in the four spectral bands (converted to ASCII) of each of the 9 pixels in the 3x3 neighbourhood and a number indicating the classification label of the central pixel. The number is a code for the following classes: 

#### Number: Class
* 1:  red soil 
* 2:  cotton crop 
* 3:  grey soil 
* 4:  damp grey soil 
* 5:  soil with vegetation stubble 
* 6:  mixture class (all types present) 
* 7:  very damp grey soil 

NB. There are no examples with class 6 in this dataset. 

The data is given in random order and certain lines of data have been removed so you cannot reconstruct the original image from this dataset. 

In each line of data the four spectral values for the top-left pixel are given first followed by the four spectral values for the top-middle pixel and then those for the top-right pixel, and so on with the pixels read out in sequence left-to-right and top-to-bottom. Thus, the four spectral values for the central pixel are given by attributes 17,18,19 and 20. If you like you can use only these four attributes, while ignoring the others. This avoids the problem which arises when a 3x3 neighbourhood straddles a boundary. 


### Attribute Information:

The attributes are numerical, in the range 0 to 255.

In [ ]:
URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/satimage/sat.trn"
canali = ["R", "G", "B", "A"]
numero_classe = {
    1: "red soil", 
    2: "cotton crop", 
    3: "grey soil", 
    4: "damp grey soil",
    5: "soil with vegetation stubble",
#    6: "mixture class",
    7: "very damp grey soil",
}
nomi = ["Pixel{}_{}".format(i, c) for i in range(9) for c in canali] + ["Classe"]

dati = pd.read_table(URL, sep=" ", names=nomi)
print("Dimensioni: {} X {}".format(*dati.shape))
dati.head()

# Analisi esplorativa

In [ ]:
plt.figure(figsize=(8, 6))

plt.suptitle("Un'immagine per classe selezionata casualmente")

for i, (num_cl, cl) in enumerate(numero_classe.items()):
    plt.subplot(231 + i)
    plt.title(cl)
    idx = np.random.choice(dati.index[dati["Classe"] == num_cl])
    image = dati.loc[idx].values[:-1].reshape((3, 3, 4)).astype(np.uint8)
    plt.imshow(image, interpolation='none')
    plt.xticks([0.5, 1.5], [])
    plt.yticks([0.5, 1.5], [])
    plt.grid(color='black')

plt.show()

In [ ]:
import seaborn as sns

In [ ]:
PIXEL = 4

plt.figure(figsize=(8, 6))

plt.suptitle("Boxplot dei pixel in posizione {} per ogni combinazione di classe e canale".format(PIXEL))

for i in range(4):
    plt.subplot(221 + i)
    canale = canali[i]
    plt.title('Canale {}'.format(canale))
    sns.boxplot(x=y_train, y=X_train["Pixel{}_{}".format(PIXEL, canale)])

plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(dati.iloc[:, :-1], dati.iloc[:, -1], 
                                                    test_size=0.25, random_state=42)

In [ ]:
X_train.describe()

In [ ]:
y_train.replace(numero_classe).value_counts()